In [7]:
import ctypes

In [8]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [9]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Person({self.name})"

    def __del__(self):
        print(f'__del__ called on {self}...')

In [10]:
p = Person('John')

__del__ called on Person(John)...


In [11]:
id_p = id(p)

In [12]:
ref_count(id_p)

1

In [13]:
p = None

__del__ called on Person(John)...


In [14]:
ref_count(id_p)

-16711708

In [15]:
p = Person('John')

In [16]:
del p

__del__ called on Person(John)...


In [17]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Person({self.name})"

    def __del__(self):
        print(f'__del__ called on {self}...')

    def gen_ex(self):
        raise ValueError('Something went bump...')

In [18]:
p = Person('Eric')
p_id = id(p)
ref_count(p_id)

1

In [21]:
try:
    p.gen_ex()
except ValueError as e:
    error = e
    print(e)

Something went bump...


In [23]:
error

ValueError('Something went bump...')

In [24]:
ref_count(p_id)

2

In [25]:
type(error)

ValueError

In [26]:
dir(error)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'add_note',
 'args',
 'with_traceback']

In [27]:
dir(error.__traceback__)

['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next']

In [28]:
dir(error.__traceback__.tb_frame)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'f_back',
 'f_builtins',
 'f_code',
 'f_globals',
 'f_lasti',
 'f_lineno',
 'f_locals',
 'f_trace',
 'f_trace_lines',
 'f_trace_opcodes']

In [29]:
type(error.__traceback__.tb_frame.f_locals)

dict

In [30]:
error.__traceback__.tb_frame.f_locals

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'def ref_count(address):\n    return ctypes.c_long.from_address(address).value',
  'class Person:\n    def __init__(self, name):\n        self.name = name\n\n    def __repr__(self):\n        return f"Person({self.name})"\n\n    def __del__(self):\n        print(f\'__del__ called on {self}...\')',
  'class Person:\n    def __init__(self, name):\n        self.name = name\n\n    def __repr__(self):\n        return f"Person({self.name})"\n\n    def __del__(self):\n        print(f\'__del__ called on {self}...\')',
  "p = Person('John')",
  'id_p = id(p)',
  'ref_count(id_p)',
  'import ctypes',
  'def ref_count(address):\n    return ctypes.c_long.from_address(address).value',
  'class Person:\n    def __init__(self, 

In [34]:
for k, v in error.__traceback__.tb_frame.f_locals.copy().items():
    if isinstance(v, Person):
        print(k, v, id(v))


p Person(Eric) 4363437008


In [35]:
ref_count(p_id)

2

In [36]:
del p

In [37]:
del error

In [38]:
ref_count(p_id)

1

In [39]:
class Person:
    def __del__(self):
        raise ValueError('Something went bump...')

In [40]:
p = Person()

In [41]:
del p

Exception ignored in: <function Person.__del__ at 0x1048fdd00>
Traceback (most recent call last):
  File "/var/folders/ct/sdgrjd_97q7g5d4q319wptzc0000gn/T/ipykernel_66594/795576010.py", line 3, in __del__
ValueError: Something went bump...


In [42]:
import sys

In [43]:
sys.stderr, sys.stdout

(<ipykernel.iostream.OutStream at 0x10407f910>,
 <ipykernel.iostream.OutStream at 0x10407f8b0>)

In [44]:
class ErrToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stderr = sys.stderr

    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stderr = self._file

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stderr = self._current_stderr
        if self._file:
            self._file.close()
        return False
        

In [45]:
p = Person()

In [46]:
with ErrToFile('err.txt'):
    del p
    print('100')
print(200)

100
200


In [47]:
with open('err.txt', 'r') as f:
    print(f.read())


Exception ignored in: <function Person.__del__ at 0x1048fdd00>
Traceback (most recent call last):
  File "/var/folders/ct/sdgrjd_97q7g5d4q319wptzc0000gn/T/ipykernel_66594/795576010.py", line 3, in __del__
ValueError: Something went bump...

